In [34]:
# Load All Necessary Packages

import os


from bs4 import BeautifulSoup
import requests
import time
import re

import pandas as pd
import numpy as np

# set options to be headless, ..
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException 

In [35]:
# open it, go to a website, and get results



options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
options.add_argument('blink-settings=imagesEnabled=false')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--headless")
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'
options.add_argument('user-agent={0}'.format(user_agent))
wd = webdriver.Chrome('chromedriver',options=options)
wd.implicitly_wait(40)

In [36]:
df=pd.read_csv('classcentral_linkedln.csv')

In [37]:
df.head()

,medium-down-width-100 href,absolute src,color-charcoal href,text-1,text-3,color-charcoal,text-3 2,text-3 3,text-3
0,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,CompTIA Network+ (N10-007) Cert Prep: 1 Unders...,2601 ratings at LinkedIn Learning,Earn your CompTIA Network+ certification. Part...,Less than 1 hour of material,On-Demand,Free Trial Available
1,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,CSS Essential Training,3699 ratings at LinkedIn Learning,Explore the concepts that form the foundation ...,4-5 hours worth of material,On-Demand,Free Trial Available
2,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,Succeeding in Web Development: Full Stack and ...,3767 ratings at LinkedIn Learning,Do you want to become a web developer? Learn a...,1-2 hours worth of material,On-Demand,Free Trial Available
3,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,JavaScript Essential Training,1 review,Get hands-on experience scripting in JavaScrip...,5 hours worth of material,On-Demand,Free Trial Available
4,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,WordPress: Workflows,174 ratings at LinkedIn Learning,Be a more efficient WordPress developer. Disco...,1-2 hours worth of material,On-Demand,Free Trial Available


In [38]:
df.columns=['urlc','img url','URL','Title','Ratings','Description','Duration','Time','Price']

In [39]:
df.head()

,urlc,img url,URL,Title,Ratings,Description,Duration,Time,Price
0,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,CompTIA Network+ (N10-007) Cert Prep: 1 Unders...,2601 ratings at LinkedIn Learning,Earn your CompTIA Network+ certification. Part...,Less than 1 hour of material,On-Demand,Free Trial Available
1,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,CSS Essential Training,3699 ratings at LinkedIn Learning,Explore the concepts that form the foundation ...,4-5 hours worth of material,On-Demand,Free Trial Available
2,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,Succeeding in Web Development: Full Stack and ...,3767 ratings at LinkedIn Learning,Do you want to become a web developer? Learn a...,1-2 hours worth of material,On-Demand,Free Trial Available
3,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,JavaScript Essential Training,1 review,Get hands-on experience scripting in JavaScrip...,5 hours worth of material,On-Demand,Free Trial Available
4,https://www.classcentral.com/course/linkedin-l...,https://d3f1iyfxxz8i1e.cloudfront.net/courses/...,https://www.classcentral.com/course/linkedin-l...,WordPress: Workflows,174 ratings at LinkedIn Learning,Be a more efficient WordPress developer. Disco...,1-2 hours worth of material,On-Demand,Free Trial Available


In [46]:
urllist=df['URL']
urllist

0       https://www.classcentral.com/course/linkedin-l...
1       https://www.classcentral.com/course/linkedin-l...
2       https://www.classcentral.com/course/linkedin-l...
3       https://www.classcentral.com/course/linkedin-l...
4       https://www.classcentral.com/course/linkedin-l...
                              ...                        
1570    https://www.classcentral.com/course/linkedin-l...
1571    https://www.classcentral.com/course/linkedin-l...
1572    https://www.classcentral.com/course/linkedin-l...
1573    https://www.classcentral.com/course/linkedin-l...
1574    https://www.classcentral.com/course/linkedin-l...
Name: URL, Length: 1575, dtype: object

In [41]:
# new_url=[]
# for url in urllist:
#         domain='https://www.classcentral.com'
#         url=url+'/visit'
#         wd.get(url)
#         time.sleep(2)
# #         bs = BeautifulSoup(wd.page_source, 'html.parser')
# #         desc=bs.find('a',id='btnProviderCoursePage')
#         get_url=''
   
# #         trim=desc['href']
# #         final=domain+trim
# #         print(final)
#         wd.get(url)
#         time.sleep(10)
#         #print(wd.page_source)
#         #bs = BeautifulSoup(wd.page_source, 'html.parser')
#         get_url = wd.current_url
 
        
#     # Printing the URL
#         print(get_url)
#         new_url.append(get_url)

KeyboardInterrupt: 

In [11]:
# df['Finalurl']=new_url

ValueError: Length of values (913) does not match length of index (1575)

In [42]:
# len(new_url)

# data=pd.DataFrame(new_url)

In [43]:
# data.head()

""


In [15]:
# data.to_csv('linked_links_913_2.csv',index=False,encoding='utf_8_sig')

In [19]:
new_url2=[]
for url in urllist[913:]:
        domain='https://www.classcentral.com'
        url=url+'/visit'
        wd.get(url)
        time.sleep(2)
#         bs = BeautifulSoup(wd.page_source, 'html.parser')
#         desc=bs.find('a',id='btnProviderCoursePage')
        get_url=''
   
#         trim=desc['href']
#         final=domain+trim
#         print(final)
        wd.get(url)
        time.sleep(10)
        #print(wd.page_source)
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        get_url = wd.current_url
 
        
    # Printing the URL
        print(get_url)
        new_url2.append(get_url)

https://www.linkedin.com/learning/deep-learning-image-recognition?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6lXAxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/ethical-hacking-scanning-networks?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6lSQxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/level-up-python?clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6n00xxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/xamarin-forms-essential-training-2?clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6n0YxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/introducing-maven?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%

https://www.linkedin.com/learning/advanced-sas-programming-for-r-users-part-2?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6iQgxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/advanced-nlp-with-python-for-machine-learning?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6iVUxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/servicenow-basic-administration?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6iV8xxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/learning-the-python-3-standard-library?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6iXIxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/oracle-java-certification-2-operators-and-decision-statements?src=aff-ref

https://www.linkedin.com/learning/building-restful-apis-with-node-js-and-express-16069959?clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6BUoxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/web-portfolio-projects-api-based-chrome-developer-tool?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6BRExxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/security-testing-nmap-security-scanning-14221942?clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6BWQxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/sas-9-4-cert-prep-part-08-controlling-data-step-processing?clickid=USHR6GQC%3AxyNWvP14g3642QWUkAR6BT0xxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: headless chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x005FDCE3]
	(No symbol) [0x005939D1]
	(No symbol) [0x004A4DA8]
	(No symbol) [0x00497FF5]
	(No symbol) [0x00497EFC]
	(No symbol) [0x00496BF6]
	(No symbol) [0x00497358]
	(No symbol) [0x004A6542]
	(No symbol) [0x004FDB91]
	(No symbol) [0x004EAECC]
	(No symbol) [0x004FD57C]
	(No symbol) [0x004EACC6]
	(No symbol) [0x004C6F68]
	(No symbol) [0x004C80CD]
	GetHandleVerifier [0x00873832+2506274]
	GetHandleVerifier [0x008A9794+2727300]
	GetHandleVerifier [0x008AE36C+2746716]
	GetHandleVerifier [0x006A6690+617600]
	(No symbol) [0x0059C712]
	(No symbol) [0x005A1FF8]
	(No symbol) [0x005A20DB]
	(No symbol) [0x005AC63B]
	BaseThreadInitThunk [0x76D100F9+25]
	RtlGetAppContainerNamedObjectPath [0x77797BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77797B8E+2

In [20]:
len(new_url2)

77

In [21]:
link=new_url+new_url2

In [22]:
len(link)

990

In [23]:
print(link)

['https://www.linkedin.com/learning/comptia-network-plus-n10-007-cert-prep-1-understanding-networks?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=z0z1d5QCFxyNRVVTAwVTuXn7UkAR-oTkxxCz2Y0&mcid=6851962469594763264&irgwc=1', 'https://www.linkedin.com/learning/css-essential-training-3?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=z0z1d5QCFxyNRVVTAwVTuXn7UkAR-oQwxxCz2Y0&mcid=6851962469594763264&irgwc=1', 'https://www.linkedin.com/learning/succeeding-in-web-development-full-stack-and-front-end?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=z0z1d5QCFxyNRVVTAwVTuXn7UkAR-oQ8xxCz2Y0&mcid=6851962469594763264&irgwc=1', 'https://www.linkedin.com/learning/javascript-essential-training?clickid=z0z1d5QCFxyNRVVTAwVTuXn7UkAR-oVcxxCz2Y0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232', 'https://www.linkedin.com/learning/wordpress-workflows-2?src=aff-ref&trk=aff-ir_p

In [24]:

dat=pd.DataFrame(link)
dat.to_csv('linked_links_913_3.csv',index=False,encoding='utf_8_sig')

In [25]:
dat.isna().sum()

0    0
dtype: int64

In [44]:
df[0:300]
df['URL'][4]

'https://www.classcentral.com/course/linkedin-learning-wordpress-workflows-100710'

In [47]:
loop = int(len(urllist) / 400)
print(loop)

df['FFFURL']=''
fflist=[]
count=0


5


In [48]:
for i in range(loop+1):
    start = i*400
    end = min((i+1)*400,len(urllist) )
    new_url2=[]
    for url in urllist[start:end]:
            #domain='https://www.classcentral.com'
            url=url+'/visit'
            wd.get(url)
            time.sleep(2)
    #         bs = BeautifulSoup(wd.page_source, 'html.parser')
    #         desc=bs.find('a',id='btnProviderCoursePage')
            get_url=''

    #         trim=desc['href']
    #         final=domain+trim
    #         print(final)
            #wd.get(url)
            time.sleep(10)
            #print(wd.page_source)
            #bs = BeautifulSoup(wd.page_source, 'html.parser')
            get_url = wd.current_url


        # Printing the URL
            print(get_url)
            new_url2.append(get_url)
            df['FFFURL'][count]=get_url
            count+=1
    fflist=fflist+new_url2
    

https://www.linkedin.com/learning/comptia-network-plus-n10-007-cert-prep-1-understanding-networks?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR6CWYxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/css-essential-training-3?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR6CTMxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/succeeding-in-web-development-full-stack-and-front-end?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR6CQsxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/javascript-essential-training?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6CVgxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/wordpress-workflows-2?src=aff-ref&trk=aff-ir_progid.8005_par

https://www.linkedin.com/learning/learning-data-analytics-1-foundations?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6pWwxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/kubernetes-microservices?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6pW8xxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/learning-java-11?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6pTIxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/building-apis-with-loopback?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR6pQsxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/introduction-to-css-14934735?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6pVExxCwwk0&mcid=6851962469594763264&src=aff-

https://www.linkedin.com/learning/cybersecurity-foundations-2020?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6rysxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/learning-functional-programming-with-javascript-es5?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6rUgxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/creating-your-first-spring-boot-microservice?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6rRQxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/python-parallel-and-concurrent-programming-part-1?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6rWwxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/ios-core-bluetooth-for-developer

https://www.linkedin.com/learning/php-testing-legacy-applications?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR6t1cxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/c-sharp-design-patterns-part-1-14140825?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6t3oxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/programming-foundations-object-oriented-design-3?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR6tyExxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/code-clinic-c-sharp-2018?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6tUQxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/cybersecurity-foundations-2020?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6tR0xxCwwk0&m

https://www.linkedin.com/learning/learning-virtualization-13945890?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6qzcxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/json-essential-training?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6qwoxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/learning-c-plus-plus-14267389?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6q1ExxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/learning-cloud-computing-serverless-computing?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6q3QxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/visual-studio-2019-essential-training?clickid=x49QhXQs0xyNWvP14g3642QWUkAR6qy

...

https://www.linkedin.com/learning/learning-github-18719601?clickid=x49QhXQs0xyNWvP14g3642QWUkAR69UMxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/introduction-to-web-design-and-development-14628245?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR69RAxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/aws-for-devops-high-availability-and-elasticity?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR69WkxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/cert-prep-comptia-linux-plus-xk0-004?clickid=x49QhXQs0xyNWvP14g3642QWUkAR69TwxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/learning-graphql-11292553?clickid=x49QhXQs0xyNWvP14g3642QWUkAR69

https://www.linkedin.com/learning/developing-aws-lambda-functions-with-kotlin?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR66ysxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.classcentral.com/course/linkedin-learning-kotlin-essential-training-39848/visit
https://www.linkedin.com/learning/dreamweaver-cs6-essential-training?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR66RQxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/learning-fedora-linux-2?clickid=x49QhXQs0xyNWvP14g3642QWUkAR66WwxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://www.linkedin.com/learning/jeffrey-zeldman-20-years-of-web-design-and-community?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAR66W8xxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin

https://www.linkedin.com/learning/linux-bash-shell-and-scripts?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0xyMxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/creating-flowcharts-for-beginners?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0xUAxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/learning-php-4?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0xRkxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/software-architecture-breaking-a-monolith-into-microservices?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0xWUxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/unity-integrating-gps-and-points-of-interest?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid

https://www.classcentral.com/course/linkedin-learning-functional-programming-with-java-76646/visit
https://www.linkedin.com/learning/software-project-management-foundations?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0zyMxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/vmware-vsphere-advanced-networking?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0zUAxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/linux-foundation-cert-prep-network-administration-ubuntu?src=aff-ref&trk=aff-ir_progid.8005_partid.259799_sid._adid.647232&clickid=x49QhXQs0xyNWvP14g3642QWUkAW0zRkxxCwwk0&mcid=6851962469594763264&irgwc=1
https://www.linkedin.com/learning/joomla-4-essential-training?clickid=x49QhXQs0xyNWvP14g3642QWUkAW0zWUxxCwwk0&mcid=6851962469594763264&src=aff-ref&irgwc=1&trk=aff-ir_progid%2E8005_partid%2E259799_sid%2E_adid%2E647232
https://w

KeyboardInterrupt: 

In [49]:
print(count)

724


In [52]:
len(new_url2)

124

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv(r'C:\Users\nehab\Desktop\Python\final datasets\LinkedIn_data.csv')

In [3]:
df.columns

Index(['urlc', 'img url', 'URL', 'Title', 'Ratings', 'Description', 'Duration',
       'Time', 'Price', 'FinalUrl', 'newRatings'],
      dtype='object')

In [4]:
urllist=df['FinalUrl']